In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('/home/jwerner/BrainPower/brainpower/submodule/')
import bp_preprocessing

#### Import the full data as a pandas dataframe, then handle the scale and NaN values as well as drop any columns which are not either the target or features

In [2]:
data_full = pd.read_csv('/home/jwerner/BrainPower/brainpower/data/unsplit_data/full_data_short.csv')

In [3]:
data_full = bp_preprocessing.handle_scale_and_nan(data_full,nandecision='drop',scale='Standard')
data_full = data_full.drop(columns='assay_ID')

#### Since this is a small dataset, use split_cats_by_tolerance to ensure that the dev and test data have equal ratios of categories

In [4]:
data_dev, data_test = bp_preprocessing.split_cats_by_tolerance(data_full,tolerance=0.01,randomstate=98281)

{'Healthy': 132, 'AD_MCI': 43, 'PD_MCI_LBD': 32, 'PD': 31}
{'Healthy': 24, 'AD_MCI': 8, 'PD_MCI_LBD': 5, 'PD': 5}
Randstate: 98281

Percent Healthy in dev, test: [0.5546218487394958, 0.5714285714285714] 
Standard deviation of these values: 0.008403361344537785 


Percent AD_MCI in dev, test: [0.18067226890756302, 0.19047619047619047] 
Standard deviation of these values: 0.004901960784313722 


Percent PD in dev, test: [0.13025210084033614, 0.11904761904761904] 
Standard deviation of these values: 0.005602240896358551 


Percent PD_MCI_LBD in dev, test: [0.13445378151260504, 0.11904761904761904] 
Standard deviation of these values: 0.007703081232492998 



#### The feature selection function works on the full dev set and returns a dataframe with statistics and selected features. The function passed in the cell below calculates a limited amount of information so that the process does not take too long. Further down are dataframes with more extensive data gathered with the following parameters:

bp_preprocessing.mrmr_feature_selection(data_dev,split=0.15,min_features=10,max_features=30,step_features=1,folds=10,ci=0.68,
                                          balancer='over_under without replacement', tolerance=0.01, target='group',
                                          model=sklearn.linear_model.RidgeClassifier(), score=sklearn.metrics.balanced_accuracy_score, featureselector=mrmr.mrmr_classif)

In [5]:
import sklearn
import mrmr

In [6]:
rforest_stats = bp_preprocessing.mrmr_feature_selection(data_dev,split=0.15,min_features=3,max_features=3,step_features=1,folds=3,ci=0.68,
                                          balancer='over_under without replacement', tolerance=0.01, target='group',
                                          model=sklearn.ensemble.RandomForestClassifier(),score=sklearn.metrics.balanced_accuracy_score,featureselector=mrmr.mrmr_classif)

100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 18.67it/s]


Time elapsed: 4.363607883453369


#### The following tables were generated this way. Because these were exported to a csv, the tuples turned into strings and must be read as literals

In [7]:
import ast

In [8]:
sys.path.append('/home/jwerner/BrainPower/data/split_data')
ridge_stats = pd.read_csv('/home/jwerner/BrainPower/brainpower/data/split_data/ridge_10_30_features.csv')
ridge_stats['ind_scores'] = ridge_stats['ind_scores'].apply(ast.literal_eval)
ridge_stats['features'] = ridge_stats['features'].apply(ast.literal_eval)

rforest_stats = pd.read_csv('/home/jwerner/BrainPower/brainpower/data/split_data/rforest_10_30_features.csv')
rforest_stats['ind_scores'] = rforest_stats['ind_scores'].apply(ast.literal_eval)
rforest_stats['features'] = rforest_stats['features'].apply(ast.literal_eval)


nbayes_stats = pd.read_csv('/home/jwerner/BrainPower/brainpower/data/split_data/nbayes_10_30_features.csv')
nbayes_stats['ind_scores'] = nbayes_stats['ind_scores'].apply(ast.literal_eval)
nbayes_stats['features'] = nbayes_stats['features'].apply(ast.literal_eval)


dectree_stats = pd.read_csv('/home/jwerner/BrainPower/brainpower/data/split_data/dectree_10_30_features.csv')
dectree_stats['ind_scores'] = dectree_stats['ind_scores'].apply(ast.literal_eval)
dectree_stats['features'] = dectree_stats['features'].apply(ast.literal_eval)

xgboost_stats = pd.read_csv('/home/jwerner/BrainPower/brainpower/data/split_data/xgboost_10_30_features.csv')
xgboost_stats['ind_scores'] = xgboost_stats['ind_scores'].apply(ast.literal_eval)
xgboost_stats['features'] = xgboost_stats['features'].apply(ast.literal_eval)

#### The features found in each run of the feature selector can be put into a dictionary by using get_feature_dict. The uniqueness parameter limits the features returned in the dictionary to only those with a multiplicity equal to or greater than the value passed or 'all' for only features which show up in every fold of the feature selection process.

#### For example, the cell below gets the feature dictionary for the 10-fold ridge classifer generated between 10 and 30 features, with only those features in each space that appear at least 3 times. of the 200 features generated by the 10 fold process, these 19 show up at least 3 times

#### The altair_feature_selection_chart function will generate charts of the feature selection information and similarly has a uniqueness parameter that functions in the same way as the dictionary. Click on a point to see the features matching the uniqueness threshold for that feature space across the 10 fold selection process

In [18]:
ridge_chart = bp_preprocessing.altair_feature_selection_chart(ridge_stats,charttitle='Ridge Classifier',uniqueness=3)
ridge_chart.save('ridge_chart.html','html')

In [19]:
rforest_chart = bp_preprocessing.altair_feature_selection_chart(rforest_stats,charttitle='Random Forest Classifier',uniqueness=3)
rforest_chart.save('rforest_chart.html','html')

In [20]:
nbayes_chart = bp_preprocessing.altair_feature_selection_chart(nbayes_stats,charttitle='Naive Bayes Classifier',uniqueness=3)
nbayes_chart.save('nbayes_chart.html','html')

In [21]:
dectree_chart = bp_preprocessing.altair_feature_selection_chart(dectree_stats,charttitle='Decision Tree Classifier',uniqueness=3)
dectree_chart.save('dectree_chart.html','html')

In [22]:
xgboost_chart = bp_preprocessing.altair_feature_selection_chart(xgboost_stats,charttitle='XGBoost Classifier',uniqueness=3)
xgboost_chart.save('xgboost_chart.html','html')